Before you begin, open this experiment on Trovi:

-   Use this link: [Large-scale model training on Chameleon](https://chameleoncloud.org/experiment/share/39a536c6-6070-4ccf-9e91-bc47be9a94af) on Trovi
-   Then, click “Launch on Chameleon”. This will start a new Jupyter server for you, with the experiment materials already in it.

You will see several notebooks inside the `llm-chi` directory - look for the one titled `1_create_server.ipynb`. Open this notebook and continue there.

## Bring up a GPU server

At the beginning of the lease time, we will bring up our GPU server. We will use the `python-chi` Python API to Chameleon to provision our server.

We will execute the cells in this notebook inside the Chameleon Jupyter environment.

Run the following cell, and make sure the correct project is selected:

In [5]:
from chi import server, context, lease
import os

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@UC")

Change the string in the following cell to reflect the name of *your* lease (**with your own net ID**), then run it to get your lease:

In [ ]:
l = lease.get_lease(f"node2-project39") # or llm_single_netID, or llm_multi_netID
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>nodes3-project…

Lease Details:
Name: nodes3-project39
ID: 643c77e1-00e5-4f9f-8ec0-07ff1cf42ce3
Status: ACTIVE
Start Date: 2025-05-10 09:10:00
End Date: 2025-05-10 15:10:00
User ID: 0ed88604379ab4845e1a1d720467e9001f097c95b423985b19619fa8f25d7f91
Project ID: 7c0a7a1952e44c94aa75cae1ff5dc9b4

Node Reservations:
ID: 5f24354b-8423-4655-9c48-b0323ab0c85c, Status: active, Min: 1, Max: 1

Floating IP Reservations:

Network Reservations:

Events:


The status should show as “ACTIVE” now that we are past the lease start time.

The rest of this notebook can be executed without any interactions from you, so at this point, you can save time by clicking on this cell, then selecting Run \> Run Selected Cell and All Below from the Jupyter menu.

As the notebook executes, monitor its progress to make sure it does not get stuck on any execution error, and also to see what it is doing!

We will use the lease to bring up a server with the `CC-Ubuntu24.04-CUDA` disk image. (Note that the reservation information is passed when we create the instance!) This will take up to 10 minutes.

In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node2-project39", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

Waiting for server node2-project39's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node2-project39
Id,2dda712d-94a7-48ea-9cbf-52144b708bb6
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.81.96 (v4) Type: fixed MAC: 14:23:f2:a4:22:c0
Network Name,sharednet1
Created At,2025-05-10T09:57:57Z
Keypair,trovi-b104fa3
Reservation Id,5f24354b-8423-4655-9c48-b0323ab0c85c
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


Note: security groups are not used at Chameleon bare metal sites, so we do not have to configure any security groups on this instance.

Then, we’ll associate a floating IP with the instance, so that we can access it over SSH.

In [8]:
s.associate_floating_ip()

In [5]:
s.refresh()
s.check_connectivity()

Checking connectivity to 192.5.87.67 port 22.


Connection successful


In [6]:
s.refresh()
s.show(type="widget")

Attribute,node2-project39
Id,2dda712d-94a7-48ea-9cbf-52144b708bb6
Status,ACTIVE
Image Name,CC-Ubuntu24.04-CUDA
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.140.81.96 (v4) Type: fixed MAC: 14:23:f2:a4:22:c0 IP: 192.5.87.67 (v4) Type: floating MAC: 14:23:f2:a4:22:c0
Network Name,sharednet1
Created At,2025-05-10T09:57:57Z
Keypair,trovi-b104fa3
Reservation Id,5f24354b-8423-4655-9c48-b0323ab0c85c
Host Id,c15c5d0cd98629a41c320d11364f137f4320899eed52f609fb88500c


## Retrieve code and notebooks on the instance

Now, we can use `python-chi` to execute commands on the instance, to set it up. We’ll start by retrieving the code and other materials on the instance.

In [7]:
s.execute("git clone --branch main https://github.com/Akkey01/MLops-Final-Project.git")

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.67: b'ee08e9465a322adeabe39294729b5701'
  warnings.warn(
Cloning into 'MLops-Final-Project'...


<Result cmd='git clone --branch main https://github.com/Akkey01/MLops-Final-Project.git' exited=0>

## Set up Docker with NVIDIA container toolkit

To use common deep learning frameworks like Tensorflow or PyTorch, we can run containers that have all the prerequisite libraries necessary for these frameworks. Here, we will set up the container framework.

In [8]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")
s.execute("docker run hello-world")

# Executing docker install script, commit: 53a22f61c0628e58e1d6680b49e82993d304b449


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin >/dev/null

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c doc

Client: Docker Engine - Community
 Version:           28.1.1
 API version:       1.49
 Go version:        go1.23.8
 Git commit:        4eba377
 Built:             Fri Apr 18 09:52:14 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.1.1
  API version:      1.49 (minimum version 1.24)
  Go version:       go1.23.8
  Git commit:       01f442b
  Built:            Fri Apr 18 09:52:14 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          1.7.27
  GitCommit:        05044ec0a9a75232cad458027ca83437aae3f4da
 runc:
  Version:          1.2.5
  GitCommit:        v1.2.5-0-g59923ef
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.


T

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
e6590344b1a5: Pulling fs layer
e6590344b1a5: Verifying Checksum
e6590344b1a5: Download complete
e6590344b1a5: Pull complete
Digest: sha256:c41088499908a59aae84b0a49c70e86f4731e588a737f1637e73c8c09d995654
Status: Downloaded newer image for hello-world:latest



Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



<Result cmd='docker run hello-world' exited=0>

We will also install the NVIDIA container toolkit, with which we can access GPUs from inside our containers.

In [9]:
# get NVIDIA container toolkit 
s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list")
s.execute("sudo apt update")
s.execute("sudo apt-get install -y nvidia-container-toolkit")
s.execute("sudo nvidia-ctk runtime configure --runtime=docker")
s.execute("sudo systemctl restart docker")

deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/stable/deb/$(ARCH) /
#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://nvidia.github.io/libnvidia-container/experimental/deb/$(ARCH) /


Get:1 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  InRelease [1477 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64  InRelease
Hit:3 https://download.docker.com/linux/ubuntu noble InRelease
Get:4 https://nvidia.github.io/libnvidia-container/stable/deb/amd64  Packages [18.6 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:6 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Components [161 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Components [376 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/restricted amd64 Components [212 B]
Get:12 http://nova.clouds.archive.ubuntu.com/ubuntu noble-u

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 5849 kB in 2s (3614 kB/s)
Selecting previously unselected package libnvidia-container1:amd64.
(Reading database ... 113813 files and directories currently installed.)
Preparing to unpack .../libnvidia-container1_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container1:amd64 (1.17.6-1) ...
Selecting previously unselected package libnvidia-container-tools.
Preparing to unpack .../libnvidia-container-tools_1.17.6-1_amd64.deb ...
Unpacking libnvidia-container-tools (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit-base.
Preparing to unpack .../nvidia-container-toolkit-base_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit-base (1.17.6-1) ...
Selecting previously unselected package nvidia-container-toolkit.
Preparing to unpack .../nvidia-container-toolkit_1.17.6-1_amd64.deb ...
Unpacking nvidia-container-toolkit (1.17.6-1) ...
Setting up nvidia-container-toolkit-base (1.17.6-1) ...
Setting up libnvidia-container1:amd64 (1.17.6-1) ...
Setting up lib

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

The processor microcode seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
time="2025-05-10T10:41:22Z" level=info msg="Config file does not exist; using empty config"
time="2025-05-10T10:41:22Z" level=info msg="Wrote updated config to /etc/docker/daemon.json"
time="2025-05-10T10:41:22Z" level=info msg="It is recommended that docker daemon be restarted."


<Result cmd='sudo systemctl restart docker' exited=0>

In the following cell, we will verify that we can see our NVIDIA GPUs from inside a container, by passing `--gpus-all`. (The `-rm` flag says to clean up the container and remove its filesystem when it finishes running.)

In [10]:
s.execute("docker run --rm --gpus all ubuntu nvidia-smi")

Unable to find image 'ubuntu:latest' locally
latest: Pulling from library/ubuntu
0622fac788ed: Pulling fs layer
0622fac788ed: Verifying Checksum
0622fac788ed: Download complete
0622fac788ed: Pull complete
Digest: sha256:6015f66923d7afbc53558d7ccffd325d43b4e249f41a6e93eef074c9505d2233
Status: Downloaded newer image for ubuntu:latest


Sat May 10 10:41:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:27:00.0 Off |                    0 |
| N/A   54C    P0             52W /  300W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

<Result cmd='docker run --rm --gpus all ubuntu nvidia-smi' exited=0>

Let’s pull the actual container images that we are going to use,

-   For the “Single GPU” section: a Jupyter notebook server with PyTorch and CUDA libraries
-   For the “Multiple GPU” section: a PyTorch image with NVIDIA developer tools, which we’ll need in order to install DeepSpeed

## Pull and start container for “Single GPU” section

Let’s pull the container:

In [15]:
s.execute("docker pull quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1")

cuda12-pytorch-2.5.1: Pulling from jupyter/pytorch-notebook
54609b48ebc1: Pulling fs layer
1bf84e16ee78: Pulling fs layer
70c528583d48: Pulling fs layer
4f4fb700ef54: Pulling fs layer
d20e2bbc3444: Pulling fs layer
e1236ab05074: Pulling fs layer
eb55310ee8e5: Pulling fs layer
247e1eb593d5: Pulling fs layer
1ac7d388a98d: Pulling fs layer
f27656f139fb: Pulling fs layer
ac0c7c38d6ad: Pulling fs layer
c48c65e32957: Pulling fs layer
d79a028c87d5: Pulling fs layer
97452dad12c2: Pulling fs layer
aad099df5a3e: Pulling fs layer
41e13f849bef: Pulling fs layer
3734b2c4ea95: Pulling fs layer
1ac7d388a98d: Waiting
5d39b32f56c0: Pulling fs layer
dfe267ba30ed: Pulling fs layer
92e65b90a905: Pulling fs layer
4f4fb700ef54: Waiting
c48c65e32957: Waiting
ac0c7c38d6ad: Waiting
bc6b6634a9a2: Pulling fs layer
35eb378f4751: Pulling fs layer
d20e2bbc3444: Waiting
f27656f139fb: Waiting
be517295261b: Pulling fs layer
d79a028c87d5: Waiting
9ae764103bf5: Pulling fs layer
e1236ab05074: Waiting
aad099df5a3e: Waitin

<Result cmd='docker pull quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1' exited=0>

and get it running:

In [16]:
s.execute("docker run -d -p 8888:8888 --gpus all --name torchnb quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1")

43936e122ad1a9ef64d7118bcc870e97585db1a57f614a63c4d4c70e6aaa9685


<Result cmd='docker run -d -p 8888:8888 --gpus all --name torchnb quay.io/jupyter/pytorch-notebook:cuda12-pytorch-2.5.1' exited=0>

## There’s one more thing we must do before we can start out Jupyter server. Rather than expose the Jupyter server to the Internet, we are going to set up an SSH tunnel from our local terminal to our server, and access the service through that tunnel.

Here’s how it works: In your *local* terminal, run

    ssh -L 8888:127.0.0.1:8888 -i ~/.ssh/id_rsa_chameleon cc@A.B.C.D

where,

-   instead of `~/.ssh/id_rsa_chameleon`, substitute the path to your key
-   and instead of `A.B.C.D`, substitute the floating IP associated with your server

This will configure the SSH session so that when you connect to port 8888 locally, it will be forwarded over the SSH tunnel to port 8888 on the host at the other end of the SSH connection.

SSH tunneling is a convenient way to access services on a remote machine when you don’t necessarily want to expose those services to the Internet (for example: if they are not secured from unauthorized access).

Finally, run

In [13]:
s.execute("docker logs torchnb")

Entered start.sh with args: start-notebook.py
Running hooks in: /usr/local/bin/start-notebook.d as uid: 1000 gid: 100
Done running hooks in: /usr/local/bin/start-notebook.d
Running hooks in: /usr/local/bin/before-notebook.d as uid: 1000 gid: 100
Sourcing shell script: /usr/local/bin/before-notebook.d/10activate-conda-env.sh
Done running hooks in: /usr/local/bin/before-notebook.d
Executing the command: start-notebook.py


[I 2025-05-10 10:46:28.200 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2025-05-10 10:46:28.203 ServerApp] jupyter_server_mathjax | extension was successfully linked.
[I 2025-05-10 10:46:28.205 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2025-05-10 10:46:28.208 ServerApp] jupyterlab | extension was successfully linked.
[I 2025-05-10 10:46:28.208 ServerApp] jupyterlab_git | extension was successfully linked.
[I 2025-05-10 10:46:28.210 ServerApp] nbclassic | extension was successfully linked.
[I 2025-05-10 10:46:28.210 ServerApp] nbdime | extension was successfully linked.
[I 2025-05-10 10:46:28.213 ServerApp] notebook | extension was successfully linked.
[I 2025-05-10 10:46:28.215 ServerApp] Writing Jupyter server cookie secret to /home/jovyan/.local/share/jupyter/runtime/jupyter_cookie_secret
[I 2025-05-10 10:46:28.368 ServerApp] notebook_shim | extension was successfully linked.
[W 2025-05-10 10:46:28.382 ServerApp] WARNING: The Jupyte

<Result cmd='docker logs torchnb' exited=0>

In [17]:
s.execute("docker logs torchnb")

Entered start.sh with args: start-notebook.py
Running hooks in: /usr/local/bin/start-notebook.d as uid: 1000 gid: 100
Done running hooks in: /usr/local/bin/start-notebook.d
Running hooks in: /usr/local/bin/before-notebook.d as uid: 1000 gid: 100
Sourcing shell script: /usr/local/bin/before-notebook.d/10activate-conda-env.sh
Done running hooks in: /usr/local/bin/before-notebook.d
Executing the command: start-notebook.py


[I 2025-05-13 09:55:33.403 ServerApp] jupyter_lsp | extension was successfully linked.
[I 2025-05-13 09:55:33.406 ServerApp] jupyter_server_mathjax | extension was successfully linked.
[I 2025-05-13 09:55:33.408 ServerApp] jupyter_server_terminals | extension was successfully linked.
[I 2025-05-13 09:55:33.411 ServerApp] jupyterlab | extension was successfully linked.
[I 2025-05-13 09:55:33.411 ServerApp] jupyterlab_git | extension was successfully linked.
[I 2025-05-13 09:55:33.413 ServerApp] nbclassic | extension was successfully linked.
[I 2025-05-13 09:55:33.413 ServerApp] nbdime | extension was successfully linked.
[I 2025-05-13 09:55:33.416 ServerApp] notebook | extension was successfully linked.
[I 2025-05-13 09:55:33.418 ServerApp] Writing Jupyter server cookie secret to /home/jovyan/.local/share/jupyter/runtime/jupyter_cookie_secret
[I 2025-05-13 09:55:33.567 ServerApp] notebook_shim | extension was successfully linked.
[W 2025-05-13 09:55:33.581 ServerApp] WARNING: The Jupyte

<Result cmd='docker logs torchnb' exited=0>

Look for the line of output in the form:

    http://127.0.0.1:8888/lab?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

and copy it for use in the next section.

You will continue working from the next notebook! But, if you are planning to do the “Multiple GPU” section in the same lease, run the following cells too, to let the container image for the “Multiple GPU” section get pulled in the background. This image takes a loooooong time to pull, so it’s important to get it started and leave it running while you are working in your other tab on the “Single GPU” section.

## Pull container for “Multiple GPU” section

In [ ]:
# s.execute("docker pull pytorch/pytorch:2.5.1-cuda12.4-cudnn9-devel")

and let’s also install some software on the host that we’ll use in the “Multiple GPU” section:

In [ ]:
# s.execute("sudo apt update; sudo apt -y install nvtop")